The purpose of this nb is to test Open Education API to get Institution info & rake-nltk for Keyword extraction

In [1]:
import requests
import io
import pandas as pd
import regex as re
import numpy as np
import json

from bs4 import BeautifulSoup
from pdfminer.high_level import extract_text
#import PyPDF2 as pdf2

import pycountry


In [2]:
%store -r date_df_ok

In [233]:
"massa of tech".title()

'Massa Of Tech'

## Affilition Retrieval:

During Affiliation Retrieval testing, it is seen that the "easiest" way to extract this information is to use a Regex capture pattern from the pdf & article.

Any other method needs either a tedious API process or possibly incorrect information.

Therefore, 2 capture patterns will be used on the article:

1- "University name" OR 2- "email@address.pt" 

If it's possible to extract the info from country code, ne ala, if not Uni. name can be processed.


In [166]:
# PART 1 - E-mail Capturing:

rec_email = "([a-zA-Z0-9._-]+@([a-zA-Z0-9._-]+\.([a-zA-Z0-9_-]+)))" 

re.findall(re.compile(rec_email, re.IGNORECASE),"bokbo aasghdjhgas merhhaba okokes@ku.edu.tr asdghjasgdash")

#mail_found = 
#cntry_code_dict[mail_found.upper()]

[('okokes@ku.edu.tr', 'ku.edu.tr', 'tr')]

In [169]:
re.search(re.compile(rec_email, re.IGNORECASE),"bokbo aasghdjhgas merhhaba okokes@ku.edu.tr asdghjasgdash").group(2)

'ku.edu.tr'

Email regex is complete!

Now to get the country info from the mail ending:

In [38]:
cntry_name_list = [cntry.name for cntry in list(pycountry.countries)]

cntry_code_dict = {cntry.alpha_2 : cntry.name for cntry in list(pycountry.countries)}

In [222]:
def from_code_to_country(pdftext):
    rec_email = "([a-zA-Z0-9._-]+@([a-zA-Z0-9._-]+\.([a-zA-Z0-9_-]+)))" 
    cntry_code_dict = {cntry.alpha_2 : cntry.name for cntry in list(pycountry.countries)}

    ccode = re.search(re.compile(rec_email, re.IGNORECASE),pdftext).group(3)
    if ccode == "com":
        cntry_found = np.nan

    else:
        if len(ccode)>2:
            ccode = ccode[:2]
        cntry_found = cntry_code_dict[ccode.upper()]

    return cntry_found


Testing:

In [223]:
# E-mail match - Option 1:

from_code_to_country(pdf_read)

#Fast BUT NOT!!! WORKING!!! for USA domains as they have no country code at the end

KeyError: 'ED'

In [219]:
# E-mail match - Option 2:


email_opt2_match = re.findall(r"(?=("+'|'.join(list(uni_code_country_dict.keys()))+r"))", pdf_read)[0]


uni_code_country_dict[email_opt2_match]

# This option REALLY slow (8-9 sec) but no need to extract any email str, loops through the entire PDF str.

'United States'

In [45]:
# E-mail match - Option 3 - BEST!:
email_reg = re.compile("([a-zA-Z0-9._-]+@([a-zA-Z0-9._-]+\.([a-zA-Z0-9_-]+)))" , re.IGNORECASE)

uni_code_country_dict[re.search(email_reg,pdf_read).group(2)]

KeyError: 'dpmms.cam.ac.uk'

In [42]:
# Country Match - Option 1:
re.findall(r"(?=("+'|'.join(cntry_name_list)+r"))", pdf_read)[0][0]

# This option is slower but it gives all the countries mentioned in mention order

'United Kingdom'

In [52]:
# Country Match - Option 2:
re.search(r"(?=("+'|'.join(cntry_name_list)+r"))", pdf_read).group(1)

'United Kingdom'

In [53]:
# Country Match - Option 3 - BEST!:
re.search(r"(?=("+'|'.join(cntry_name_list)+r"))", pdf_read)[1]

'United Kingdom'

In [43]:
# Country match - Option 4: 
str_match = [s for s in cntry_name_list if s in pdf_read]
print(str_match)

# This option is faster but it gives all the countries mentioned in alphabetical order

['Belgium', 'Canada', 'Germany', 'France', 'United Kingdom', 'Georgia', 'India', 'Israel', 'Japan']


In [44]:
#Country match - Option 5:
print(next((x for x in cntry_name_list if x in pdf_read), False))

Belgium


In [ ]:
# University Match - option 1:

# same as country match opt 1, join all uni names & search in whole text

In [ ]:
# University Match - option 2:

# same as email match opt 3, create a regex pattern to capture uni name & the use dict or join keys to only check the captured parts

## Creating the Affiliation Function:


In [14]:
# Get a random pdf_read:

random_dir_url = date_df_ok.direct_url.sample(1).values[0]

HEADERS = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36'}

pdf_req = requests.get(random_dir_url,headers=HEADERS)

print(pdf_req)
pdf_io = io.BytesIO(pdf_req.content)
pdf_read = extract_text(pdf_io,page_numbers=[])

print(random_dir_url)


<Response [200]>
https://sci.bban.top/pdf/10.1007/s10658-010-9624-y.pdf#view=FitH


In [3]:
# Creating necessary dicts & lists:

with open('world_universities_and_domains.json', encoding="utf8") as fp:
    uni_dict = json.load(fp)

uni_df = pd.DataFrame(uni_dict)

uni_code_country_dict = {}
uni_name_country_dict = {}

for index, row in uni_df.iterrows():
    uni_code_country_dict.update({dom : row.country for dom in row.domains})
    uni_name_country_dict.update({row["name"] : row.country})

del uni_name_country_dict["University of Technology"]

uni_name_list = list(uni_name_country_dict.keys())


In [4]:
# Creating necessary dicts & lists (cont.):

#email_reg_original = re.compile("([a-zA-Z0-9._-]+@([a-zA-Z0-9._-]+\.([a-zA-Z0-9_-]+)))" , re.IGNORECASE)

email_reg_modified = re.compile("[a-zA-Z0-9._-]+@([a-zA-Z0-9_-]+\.((?:[a-zA-Z0-9_-]+\.)*([a-zA-Z0-9_-]+)))")

cntry_name_list = [cntry.name for cntry in list(pycountry.countries)]
cntry_name_list.append("USA")

cntry_name_list_all_caps = [cntry.name.upper() for cntry in list(pycountry.countries)]

cntry_code_dict = {cntry.alpha_2 : cntry.name for cntry in list(pycountry.countries)}

In [34]:
pdf__demo_read = "bokbok bashdjas asdhas okokes@ku.edu.tr askdhjaskhds"

pdf_fail_read = "ahsjdkhasd asjkdashjdh lgjknf asjdhjas"

In [5]:
# 1- E-Mail(Uni Domain) - DONE! :

def email_converter(pdf_read):
    email_caught = re.search(email_reg_modified,pdf_read)

    try:
        return uni_code_country_dict[email_caught.group(1).lower()]
    except AttributeError:
        return np.nan
    except KeyError:
        try:
            return uni_code_country_dict[email_caught.group(2).lower()]
        except KeyError:
            try:
                return cntry_code_dict[email_caught.group(3).upper()]
            except KeyError:
                cntry_pttrn =  email_caught.group(3)[:3].upper()

                if cntry_pttrn in ["COM","NET"]:
                    return np.nan

                elif cntry_pttrn in ["EDU","GOV"]:
                    return "United States"

                if len(cntry_pttrn)==3:
                    cntry_pttrn = cntry_pttrn[:-1]

                try:
                    return cntry_code_dict[cntry_pttrn]
                except:
                    return np.nan


In [19]:
email_converter(pdf_read)

nan

In [6]:
# 2- Country Match - DONE:
def country_converter(pdf_read):
    try:
        cntry_match = re.search(r"(?=("+'|'.join(cntry_name_list)+r"))", pdf_read)[1]
        return  cntry_match if cntry_match != "USA" else "United States"
    except TypeError:
        try:             
            return re.search(r"(?=("+'|'.join(cntry_name_list_all_caps)+r"))", pdf_read)[1].title()
        except:
            return np.nan

In [188]:
country_converter(pdf_read)

'United Kingdom'

In [249]:
"Scotland" in cntry_name_list

False

In [7]:
# 3- University Match - DONE! :
def univ_converter(pdf_read):   
    try:
        uni_found = re.search(r"(?=("+'|'.join(uni_name_list)+r"))", pdf_read)[1]
        return uni_name_country_dict[uni_found]
    except:
        return np.nan

In [202]:
univ_converter(pdf_read)

'United Kingdom'

All three Affiliation capture funcitons are ready! Required variables are:

* "World Unis" json & pycountry
* E-mail fn:
    - email_reg_modified
    - uni_code_country_dict
    - country_code_dict
* Country fn:
    - cntry_name_list
    - cntry_name_list_all_caps
* Uni fn:
    - uni_name_list
    - uni_name_country_dict

---


### Notes on Affiliation Capture (OLD notes, taken care of in the final version)

MUST FIX Observations from Tests:

* .gmail.com emails -> COLOMBIA!
* "USA" not in country list
* email & country_match give out different results in some cases, would be very useful to have a third "University" field

Other Noteworthy Observations:
* Not all authors have email info, but have address written!
* Not all first authors are working in a "University", but rather in a "Institute"

---


World universities JSON offers info on Uni name, domain name and  country for all universities worldwide. This package therefore might be a useful for both e-mail capture & uni name capture. It will be tested below: 

In [155]:
with open('world_universities_and_domains.json') as fp:
    uni_dict = json.load(fp)

uni_df = pd.DataFrame(uni_dict)

uni_code_country_dict = {}
uni_name_country_dict = {}

for index, row in uni_df.iterrows():
    uni_code_country_dict.update({dom : row.country for dom in row.domains})
    uni_name_country_dict.update({row["name"] : row.country})


In [213]:
uni_name_country_dict["Koç University"]

'Turkey'

In [11]:
uni_code_country_dict["ku.edu.tr"]

'Turkey'

In [12]:
uni_code_country_dict["unl.pt"]

'Portugal'

In [218]:
uni_code_country_dict["umn.edu"]

'United States'

The code below shows that there are uni_codes that are seen multiple times, causing the length of the dict to be different than total codes in the JSON. One such example is rutgers.edu:

In [192]:
#from collections import Counter

#unlist =uni_df.domains.tolist()
#flatun = [item for sublist in unlist for item in sublist]
#Counter(flatun,a)

#uni_df[uni_df.domains.map(lambda x: "rutgers.edu" in x)]

### Scholarly - Affiliation Retr. Tests:

In [16]:
from scholarly import scholarly, ProxyGenerator

pg = ProxyGenerator()
success = pg.FreeProxies()
scholarly.use_proxy(pg)

search_qry = scholarly.search_author('Steven A Cholewiak')

author = next(search_qry)


In [17]:
author

{'container_type': 'Author',
 'filled': [],
 'source': <AuthorSource.SEARCH_AUTHOR_SNIPPETS: 'SEARCH_AUTHOR_SNIPPETS'>,
 'scholar_id': '4bahYMkAAAAJ',
 'url_picture': 'https://scholar.google.com/citations?view_op=medium_photo&user=4bahYMkAAAAJ',
 'name': 'Steven A. Cholewiak, PhD',
 'affiliation': 'Vision Scientist at Google LLC',
 'email_domain': '@google.com',
 'interests': ['Depth Cues',
  '3D Shape',
  'Shape from Texture & Shading',
  'Naive Physics',
  'Haptics'],
 'citedby': 399}

In [18]:
search_qry = scholarly.search_author("yan dongpeng")
author = next(search_qry)

In [22]:
author

{'container_type': 'Author',
 'filled': [],
 'source': <AuthorSource.SEARCH_AUTHOR_SNIPPETS: 'SEARCH_AUTHOR_SNIPPETS'>,
 'scholar_id': 'CDfzddYAAAAJ',
 'url_picture': 'https://scholar.google.com/citations?view_op=medium_photo&user=CDfzddYAAAAJ',
 'name': 'Dongpeng Yan',
 'affiliation': 'Beijing Normal University, Beijing University of Chemical Technology',
 'email_domain': '@bnu.edu.cn',
 'interests': ['Materials Chemistry',
  'Molecule Science',
  'Chemical Engineering'],
 'citedby': 11105}

In [29]:
date_df_ok.loc["10.1002/aic.12400","subject"]

['General Chemical Engineering', 'Environmental Engineering', 'Biotechnology']

In [30]:
any(intr in " ".join(date_df_ok.loc["10.1002/aic.12400","subject"]) for intr in author["interests"])

True

In [28]:
any(x in author["interests"] for x in date_df_ok.loc["10.1002/aic.12400","subject"])

False

In [14]:
next(sm_test)

{'container_type': 'Author',
 'filled': [],
 'source': <AuthorSource.SEARCH_AUTHOR_SNIPPETS: 'SEARCH_AUTHOR_SNIPPETS'>,
 'scholar_id': 'CDfzddYAAAAJ',
 'url_picture': 'https://scholar.google.com/citations?view_op=medium_photo&user=CDfzddYAAAAJ',
 'name': 'Dongpeng Yan',
 'affiliation': 'Beijing Normal University, Beijing University of Chemical Technology',
 'email_domain': '@bnu.edu.cn',
 'interests': ['Materials Chemistry',
  'Molecule Science',
  'Chemical Engineering'],
 'citedby': 11105}

## Abstract Retrieval Tests:

### 1- Using Science Parse:

In [1]:
from science_parse_api.api import parse_pdf
import pprint

In [2]:
from pathlib import Path
import tempfile

In [36]:
date_df_ok.loc["10.1002/aic.14601","direct_url"]

'https://sci.bban.top/pdf/10.1002/aic.14601.pdf#view=FitH'

In [12]:
pdf_req = requests.get("https://sci.bban.top/pdf/10.1002/aic.14601.pdf#view=FitH")
pdf_io = io.BytesIO(pdf_req.content)

In [13]:
   # If it cannot find that folder will get the pdf and 
    # image from Github. This will occur if you are using 
    # Google Colab
    #pdf_url = ('https://github.com/UCREL/science_parse_py_api/'
    #           'raw/master/test_data/example_for_test.pdf')

temp_test_pdf_paper = tempfile.NamedTemporaryFile('rb+')
test_pdf_paper = Path(temp_test_pdf_paper.name)
temp_test_pdf_paper.write(pdf_req.content)


10585

In [17]:
temp_test_pdf_paper.close()

In [18]:
temp_test_pdf_paper

In [15]:
test_pdf_paper

WindowsPath('C:/Users/oguzk/AppData/Local/Temp/tmpfm66jlm_')

In [14]:
extract_text(temp_test_pdf_paper.con)

TypeError: Unsupported input type: <class 'tempfile._TemporaryFileWrapper'>

In [3]:
test2_path = Path(r"C:\Users\oguzk\NOVA\THEsis\Codes\main1.pdf")

In [16]:
host = 'http://127.0.0.1'
port = '8080'
output_dict = parse_pdf(host, test_pdf_paper, port=port)

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(output_dict)

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\oguzk\\AppData\\Local\\Temp\\tmpfm66jlm_'

In [ ]:
with test_pdf_paper.open('r') as test_fp:
    test_fp.write(pdf_req.content)

In [32]:
output_dict

{'id': 'empty'}

### 2- Using rake_nltk

There are two possible solutions to the keywords problem. One is a manual regexx pattern capture & the othehr being rake-nltk or a similar library. Depending on the performance of each one will be selected & added to sh.get_dates() pipeline

In [30]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [5]:
from rake_nltk import Rake

In [27]:
#PDF_Miner

with open("BiblTest\main1.pdf", 'rb') as f:
    pdf_six = extract_text(f,page_numbers=[]).replace("\n", "")

In [21]:
#PyPDF2

with open("BiblTest\main1.pdf", 'rb') as f2:
    pdf_pdf = pdf2.PdfFileReader(f2)
    page = pdf_pdf.getPage(0)
    page_text = page.extractText()
    r.extract_keywords_from_text(page_text)
    #print(page.extractText())


In [32]:
pdf_six_sent_tokens = sent_tokenize(pdf_six)

In [38]:
r = Rake()
r.extract_keywords_from_sentences(pdf_six_sent_tokens)
r.get_ranked_phrases_with_scores()

[(193.1721852014278,
  '25 journals 7 journals 6 journals 3 social science journals 3 natural science journals one journal15 journals10 journals one journal 7 journals 14 journals 28 commercial'),
 (130.0310584070018,
  '19971992 1985 – 1999 1997 2002 2004 2004 2005 2009 economics econometrics statistics econ ., management physics'),
 (81.31818181818181,
  'ac ceptedche mistry engin eering biomedicine physics earth science mathema'),
 (71.49940982682381, '1577 /$ – see front matter © http :// dx'),
 (69.82727272727273,
  'analytical chemistry geoscience mainly engineering agriculture biomedicine civil engineering cross'),
 (61.21734410198315,
  'two conventional 26 iranian journals 1980 1986 – 1990 1994'),
 (51.1, 'revised form 3 september 2013accepted 4 september 2013keywords'),
 (47.56033724340177,
  'acceptdiscipline journal size journal article × size discipline total 3'),
 (44.26774465080917,
  'publishdiscipline journal size journal article × size discipline total 5'),
 (42.66953

In [19]:
r = Rake()
r_kw = r.extract_keywords_from_text(pdf)
r.get_ranked_phrases()[:10]

['25 journals 7 journals 6 journals 3 social science journals 3 natural science journals one journal15 journals10 journals one journal 7 journals 14 journals 28 commercial',
 '19971992 1985 – 1999 1997 2002 2004 2004 2005 2009 economics econometrics statistics econ ., management physics',
 'ac ceptedche mistry engin eering biomedicine physics earth science mathema',
 '1577 /$ – see front matter © http :// dx',
 'analytical chemistry geoscience mainly engineering agriculture biomedicine civil engineering cross',
 'two conventional 26 iranian journals 1980 1986 – 1990 1994',
 'revised form 3 september 2013accepted 4 september 2013keywords',
 'acceptdiscipline journal size journal article × size discipline total 3',
 'publishdiscipline journal size journal article × size discipline total 5',
 'publishingin open access journals (“ gold oa ”)']

In [36]:
r.extract_keywords_from_text("The publishing delay in scholarly peer-reviewed journals")

In [37]:
r.get_ranked_phrases_with_scores()

[(4.0, 'scholarly peer'),
 (4.0, 'reviewed journals'),
 (4.0, 'publishing delay')]

In [17]:
pdf_pdf.getPage(0)

ValueError: seek of closed file

In [9]:
r_kw

In [7]:
len(pdf_token)

8360

In [11]:
"References" in pdf_token

False

### 3- Creating a manual Regex pattern

In [8]:
def rec_date_capture(pdf_text):

    # REGEX PATTERNS - RECEIVED & SUBMITTED
    # 1 - 10 MAY 1960:
    rec_1 =  "(?:(?:received|submitted)\s?(?:on|date)?\s?\:?\s?)(\d{1,2}\s*(Jan(uary|.)?|Feb(ruary|.)?|Mar(ch|.)?|Apr(il|.)?|May|Jun(e|.)?|Jul(y|.)?|Aug(ust|.)?|Sep(tember|.)?|Oct(ober)?|Nov(ember|.)?|Dec(ember|.)?)\s*(19|20)\d{2})"
    # 2 - MAY 10, 1960
    rec_2 = "(?:(?:received|submitted)\s?(?:on|date)?\s?\:?\s?)((Jan(uary|.)?|Feb(ruary|.)?|Mar(ch|.)?|Apr(il|.)?|May|Jun(e|.)?|Jul(y|.)?|Aug(ust|.)?|Sep(tember|.)?|Oct(ober|.)?|Nov(ember|.)?|Dec(ember|.)?)\s?\d{1,2}\,?\s?(19|20)\d{2})"
    # 3 - MAY 1960
    rec_3 = "(?:(?:received|submitted)\s?(?:on|date)?\s?\:?\s?)((Jan(uary|.)?|Feb(ruary|.)?|Mar(ch|.)?|Apr(il|.)?|May|Jun(e|.)?|Jul(y|.)?|Aug(ust|.)?|Sep(tember|.)?|Oct(ober|.)?|Nov(ember|.)?|Dec(ember|.)?)\s*(19|20)\d{2})"
    # 4 - 10/05/1960 OR 10.05.1960 OR 10/05/1960
    rec_4 = "(?:(?:received|submitted)\s?(?:on|date)?\s?\:?\s?)([1-3][0-9](\.|\-|\/|\:)?[0-1][0-9](\.|\-|\/\:)?(19|20)\d{2})"

    rec_patterns = [rec_1, rec_2, rec_3, rec_4]

    for pattern in rec_patterns:
        comp_pattern = re.compile(pattern, re.IGNORECASE) 
        try:
            date = re.search(comp_pattern,pdf_text).group(1)
        except AttributeError:
            pass
        else:
            return date

In [236]:
# Get a random pdf_read:

#random_dir_url = date_df_ok[date_df_ok.Rec_date.notna()].direct_url.sample(1).values[0]

HEADERS = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36'}

pdf_req = requests.get(random_dir_url,headers=HEADERS)

print(pdf_req)
pdf_io = io.BytesIO(pdf_req.content)
pdf_read = extract_text(pdf_io,page_numbers=[])

print(random_dir_url)


<Response [200]>
https://www.nature.com/articles/eye201520.pdf


In [235]:
random_dir_url = "https://www.nature.com/articles/eye201520.pdf"

In [237]:
pdf_read = pdf_read.replace("\n"," ")
pdf_read = re.sub(' +', ' ', pdf_read)

In [242]:
email_reg_modified.search(pdf_read)

In [243]:
re.search(email_reg_modified,pdf_read)

In [246]:
print(f"Email: {email_converter(pdf_read)}")

print(f"Country: {country_converter(pdf_read)}")

print(f"University: {univ_converter(pdf_read)}")



Email: nan
Country: United States
University: nan


In [230]:
print(rec_date_capture(pdf_read))

May 2009


### 4- Preprocessing for Keyword Extraction:

During the analysis, it is seen that the best (most efficient) method for Content Analysis will be a manual preprocc. & rake_nltk.

Fortunately, as the "text"s in question are scientific articles, not a lot of preproc. is necessary, but still required. These are determined as:

- Remove:
    - e-mail, website links, date, number, paranthesis, brackets, "Figure ..."
    - stopwords
    - any word w/ len: <2 & >20
- Slice after "References" or "Acknowledgements" (but from end to begin.)
- lowercase all & word_tokenizer
- Lemmatize

---

Observations:

* rake-nltk seems to perform quite poorly!
* KeyBERT seems like a good potential tool for the project!



In [39]:
from nltk.corpus import stopwords
from nltk.corpus import words

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import snowballstemmer

from keybert import KeyBERT


In [231]:
# NLP - Prepocessing:

#pdf_read = pdf_read.replace("\n"," ")
#pdf_read = re.sub(' +', ' ', pdf_read)

# Run get_dates()
# Run get_affil()

#wordnet_lemmatizer = WordNetLemmatizer()
#stemmer = snowballstemmer.stemmer('english')
#kw_model = KeyBERT()

#stopwords_set = set(stopwords.words("english"))
#words_set = set(words.words())

# Lowercase all:
pdf_modified = pdf_read.lower()

# Removing "References":
pdf_modified = "".join(pdf_modified.split("references")[:-1])

# Remove URL:
pdf_modified = re.sub(r'http\S+', '', pdf_modified)

# Remove emails:
pdf_modified = re.sub(r"\S*@\S*\s?", "", pdf_modified)

# Remove everything in brackets & paranthesis:
pdf_modified = re.sub("[\(\[].*?[\)\]]", "", pdf_modified)

# Remove punctuation:
pdf_modified = re.sub(r'[^\w\s]', '', pdf_modified)

# Remove numbers
pdf_modified = re.sub(r'[0-9]', '', pdf_modified)

# Remove HTML tags:
pdf_modified = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\xff]', '', pdf_modified)

# Remove single letters:
pdf_modified = re.sub(r"(?<=(^|\s))\D(\s|$)","", pdf_modified)

# Remove multiple spaces:
pdf_modified = re.sub(' +', ' ', pdf_modified)

# Tokenize:
#pdf_modified = word_tokenize(pdf_modified)

# Simple Lemmatization Trial:
#pdf_modified = " ".join([wordnet_lemmatizer.lemmatize(word) for word in pdf_modified])

# Removing Stopwords & <2 - >15 words & Lemmatizing:
#pdf_modified = [wordnet_lemmatizer.lemmatize(word) for word in pdf_modified if word not in stopwords_set if word in words_set if len(word)>2 if len(word)<15]

# Re-merging the text:
#pdf_modified = " ".join(pdf_modified)


In [232]:
keywords = kw_model.extract_keywords(pdf_modified ,keyphrase_ngram_range=(1, 1), stop_words="english", top_n=10)
print(keywords)

kw_lemma = [wordnet_lemmatizer.lemmatize(word) for word, sc in keywords]
kw_stem = [stemmer.stemWord(word) for word, sc in keywords]
kw_lemmastem = [stemmer.stemWord(word) for word in kw_lemma]

print(f"Lemma: {set(kw_lemma)} - len: {len(set(kw_lemma))}")
print(f"Stem: {set(kw_stem)} - len: {len(set(kw_stem))}")
print(f"LemmaStem: {set(kw_lemmastem)} - len: {len(set(kw_lemmastem))}")


[('volatility', 0.3955), ('varianceoptimal', 0.3739), ('volatil', 0.367), ('leverage', 0.3311), ('semimartingale', 0.3199), ('variance', 0.3083), ('levy', 0.3049), ('variation', 0.2992), ('stochastic', 0.2828), ('stock', 0.2822)]
Lemma: {'stock', 'leverage', 'variation', 'varianceoptimal', 'semimartingale', 'levy', 'stochastic', 'variance', 'volatil', 'volatility'} - len: 10
Stem: {'stock', 'levi', 'leverag', 'semimartingal', 'variat', 'varianc', 'varianceoptim', 'volatil', 'stochast'} - len: 9
LemmaStem: {'stock', 'levi', 'leverag', 'semimartingal', 'variat', 'varianc', 'varianceoptim', 'volatil', 'stochast'} - len: 9


In [224]:
# Tokenize:
pdf_modified = word_tokenize(pdf_modified)

# Simple Lemmatization Trial:
pdf_modified = " ".join([wordnet_lemmatizer.lemmatize(word) for word in pdf_modified])

In [225]:
keywords = kw_model.extract_keywords(pdf_modified ,keyphrase_ngram_range=(1, 1), stop_words="english", top_n=10)
print(keywords)

kw_lemma = [wordnet_lemmatizer.lemmatize(word) for word, sc in keywords]
kw_stem = [stemmer.stemWord(word) for word, sc in keywords]
kw_lemmastem = [stemmer.stemWord(word) for word in kw_lemma]

print(f"Lemma: {set(kw_lemma)} - len: {len(set(kw_lemma))}")
print(f"Stem: {set(kw_stem)} - len: {len(set(kw_stem))}")
print(f"LemmaStem: {set(kw_lemmastem)} - len: {len(set(kw_lemmastem))}")


[('recombination', 0.4157), ('recombinationa', 0.3894), ('recombinationb', 0.3833), ('schizosaccharomyces', 0.3808), ('recombinant', 0.3778), ('recombinasemediated', 0.37), ('gene', 0.3673), ('mutation', 0.3494), ('genotype', 0.3489), ('recombina', 0.3476)]
Lemma: {'recombinasemediated', 'schizosaccharomyces', 'gene', 'mutation', 'recombination', 'recombinationa', 'genotype', 'recombinationb', 'recombinant', 'recombina'} - len: 10
Stem: {'schizosaccharomyc', 'recombin', 'mutat', 'gene', 'genotyp', 'recombinasemedi', 'recombinationa', 'recombinationb', 'recombina'} - len: 9
LemmaStem: {'schizosaccharomyc', 'recombin', 'mutat', 'gene', 'genotyp', 'recombinasemedi', 'recombinationa', 'recombinationb', 'recombina'} - len: 9


In [138]:
keywords2 = kw_model.extract_keywords(pdf_modified ,keyphrase_ngram_range=(1, 2), stop_words="english")
print(keywords2)

[('research tourism', 0.6241), ('tourism research', 0.6149), ('tourism researchers', 0.5943), ('tourism study', 0.5822), ('communities tourism', 0.5771)]


In [104]:
keywords3 = kw_model.extract_keywords(pdf_modified ,keyphrase_ngram_range=(1, 1), use_maxsum=True)
print(keywords3)

[('income', -0.0462), ('poor', 0.1332), ('governance', -0.0135), ('constituency', 0.1646), ('oligarchic', 0.0591)]


In [103]:
keywords4 = kw_model.extract_keywords(pdf_modified ,keyphrase_ngram_range=(1, 2), use_maxsum=True)
print(keywords4)

[('income wealthy', -0.0106), ('poor democracy', 0.0501), ('redistribution poor', 0.0787), ('oligarchy ing', 0.0689), ('research oligarchy', 0.0611)]


In [107]:
keywords5 = kw_model.extract_keywords(pdf_read, keyphrase_ngram_range=(1, 2))
print(keywords5)

[('democracy oligarchy', 0.6004), ('eu oligarchy', 0.5918), ('oligarchy representativeness', 0.5473), ('oligarchy unequal', 0.5394), ('oligarchy politics', 0.5258)]


In [106]:
keywords6 = kw_model.extract_keywords(pdf_read, keyphrase_ngram_range=(1, 2), use_maxsum=True)
print(keywords6)

[('democratic representation', 0.06), ('governance issn', 0.0372), ('government european', 0.1113), ('article democracy', 0.1076), ('oligarchy poverty', 0.1695)]


KeyBERT has the best potential for being the keyword extractor. However, it takes a REALLY (9-10 secs) for it to run on the whole pdf_read.

The alternative version, pdf_modified, is faster for Keybert, but the preprocessing itself takes around 9-19 seconds, so the overall process is actualy even longer!

As a potential "middle ground" between the two, preprocessing can be scraped almost completely, except for the "References slicing". This will be tested below: 

In [114]:
# Removing "References":
pdf_simple_mod = "".join(pdf_read.lower().split("references")[:-1])
pdf_simple_mod = re.sub(' +', ' ', pdf_simple_mod)